In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:

df = pd.read_csv('sphist.csv')
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values('Date', ascending = True)
df.head(5)



,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


### Generating indicators: 
I am going to add three indicators to the data set:

* The average price from the past 5 days.
* The average price for the past 30 days.
* The average price for the past 365 days.

In [3]:

def add_mean_indicator_col(df, num_days, indicator_name, col, function):
    #Make a series of Close price with the dates as indexes
    s = pd.Series(np.array(df[col]), index=np.array(df["Date"]))
    #calculate the mean price of past days
    means = s.rolling(window = num_days).apply(function)
    
    #Shift indices to exclude the price of each day from the mean value
    means = means.shift()
    
    #convert indices to Date Column
    means = means.reset_index()
    means = means.rename(columns={'index':'Date', 0:indicator_name})
    
    df_new = df.merge(means, left_on='Date', right_on='Date')
    return df_new

df = add_mean_indicator_col(df, 5, 'mean_close_5', 'Close', np.mean)
df = add_mean_indicator_col(df, 30, 'mean_close_30', 'Close', np.mean)
df = add_mean_indicator_col(df, 365, 'mean_close_365', 'Close', np.mean)
print('head:\n', df.head(7))
print('tail:\n', df.tail(5))

head:
         Date       Open       High        Low      Close     Volume  \
0 1950-01-03  16.660000  16.660000  16.660000  16.660000  1260000.0   
1 1950-01-04  16.850000  16.850000  16.850000  16.850000  1890000.0   
2 1950-01-05  16.930000  16.930000  16.930000  16.930000  2550000.0   
3 1950-01-06  16.980000  16.980000  16.980000  16.980000  2010000.0   
4 1950-01-09  17.080000  17.080000  17.080000  17.080000  2520000.0   
5 1950-01-10  17.030001  17.030001  17.030001  17.030001  2160000.0   
6 1950-01-11  17.090000  17.090000  17.090000  17.090000  2630000.0   

   Adj Close  mean_close_5  mean_close_30  mean_close_365  
0  16.660000           NaN            NaN             NaN  
1  16.850000           NaN            NaN             NaN  
2  16.930000           NaN            NaN             NaN  
3  16.980000           NaN            NaN             NaN  
4  17.080000           NaN            NaN             NaN  
5  17.030001        16.900            NaN             NaN  
6  1

## adding others indicators to improve the error : 
With only 3 indicators we had a rmse of 22.22

In [4]:
#Adding the average volume over the 5 past days
df = add_mean_indicator_col(df, 5, 'avg_volume_5', 'Volume', np.mean)

#Adding the standard deviation of the average volume over the past five days.
df=add_mean_indicator_col(df, 5, 'std_dev_volume_5', 'Volume', np.std)



In [5]:
#The ratio between the average price for the past 5 days, and the average price for the past 365 days.

df['ratio_avg_close']=(df['mean_close_365'])/(df['mean_close_5'])


#The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.
#Adding the standard deviation of the price over the past 365 and 5 days.
df = add_mean_indicator_col(df, 5, 'std_close_5', 'Close', np.std)
df = add_mean_indicator_col(df, 365, 'std_close_365', 'Close', np.std)
#the ratio
df['ratio_std_close'] = df['std_close_5']/df['std_close_365']

df

,Date,Open,High,Low,Close,Volume,Adj Close,mean_close_5,mean_close_30,mean_close_365,avg_volume_5,std_dev_volume_5,ratio_avg_close,std_close_5,std_close_365,ratio_std_close
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1.260000e+06,16.660000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1.890000e+06,16.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2.550000e+06,16.930000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2.010000e+06,16.980000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2.520000e+06,17.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16585,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2087.024023,2073.984998,2035.531178,3.207544e+09,9.834384e+08,0.975327,3.502675,64.282022,0.054489
16586,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2090.231982,2076.283993,2035.914082,3.232372e+09,9.942372e+08,0.974013,7.116786,64.264312,0.110742
16587,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2088.306006,2077.908659,2036.234356,3.245514e+09,1.003170e+09,0.975065,8.348225,64.189442,0.130056
16588,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.456006,2078.931331,2036.507343,3.536224e+09,1.056480e+09,0.978875,17.530725,64.033724,0.273773


## Splitting up the data and cleaning data


In [6]:
##### Remove any rows from the DataFrame that fall before `1951-01-03`.
filter=df["Date"]>datetime(year=1951,month=1,day=3)
df_clean=df[filter]
                       
#####dropna: 
df_clean=df_clean.dropna(axis=0)


In [7]:
df_clean.isnull().sum()

Date                0
Open                0
High                0
Low                 0
Close               0
Volume              0
Adj Close           0
mean_close_5        0
mean_close_30       0
mean_close_365      0
avg_volume_5        0
std_dev_volume_5    0
ratio_avg_close     0
std_close_5         0
std_close_365       0
ratio_std_close     0
dtype: int64

## Making Predictions

In [ ]:
def train_and_test(df, features,target):
    train  = df[df["Date"] < datetime(year=2013, month=1, day=1)]
    test = df[df["Date"] >= datetime(year=2013, month=1, day=1)]
    #initialize model
    lr = LinearRegression()

    #Train
    lr.fit(train[features], train[target])

    #Test
    predictions = lr.predict(test[features])

    #Calculate error
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
features=['mean_close_5', 'mean_close_30', 'mean_close_365','avg_volume_5','std_dev_volume_5','std_close_5','std_close_365','ratio_avg_close','ratio_std_close']
target=['Close']
train_and_test(df_clean,features,target)

## Add new indicators in order to reduce the error ! 

>Using the average values and the standard deviation of the past days of the Volume column does not show improvement in prediction. 
* rmse=22.28188272136987.
il n'y a pas une grand amélioration. 

>we will add new indicators to see if there are improvements. 
* The ratio between the average price for the past 5 days, and the average price for the past 365 days.
* The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.
* rsme=22.29
encore une fois, il n'y a pas une grande amélioration. 

## Last improvements: predictions only one day ahead. 
Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.



In [8]:
def train_and_test_1day(df, features,target):
    train  = df[df["Date"] < datetime(year=2013, month=1, day=3)]
    test = df[df["Date"] == datetime(year=2013, month=1, day=4)]
    #initialize model
    lr = LinearRegression()

    #Train
    lr.fit(train[features], train[target])

    #Test
    predictions = lr.predict(test[features])

    #Calculate error
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    return rmse
    

In [10]:
features=['mean_close_5', 'mean_close_30', 'mean_close_365','avg_volume_5','std_dev_volume_5','std_close_5','std_close_365','ratio_avg_close','ratio_std_close']
target=['Close']
train_and_test_1day(df_clean,features,target)


32.69042267282157